In [1405]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [1406]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)


In [1407]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = np.mean(np.exp(local_series))
            median_linear = np.median(np.exp(local_series))
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, "median_Ne_linear": median_linear, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [1408]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
            next_interval = int(interval)+3 #averaging over two weeks to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
           
            try:
                new_df["Ne"+ ".diff." + str(interval)] = (52/5)*(input_df[i].astype("float") - input_df["Ne"+"." + str(next_interval)].astype('float'))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [1409]:
def generate_local_and_intro_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/14
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            new_df["rt" +  "." + str(interval)] = (1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))
            

            
    return(new_df)

In [1410]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    count = 0
    for i in input_df.columns.tolist():
        #print(i)
        if "rt" in i:
            if count %1 == 0:
                interval = i.split(".")[1]
                #interval = i.split(".")[2]
                local_series = input_df[i].astype('float').to_numpy()
                mean_percent = local_series.mean()
                median_percent = np.median(local_series)

                hpd_95 = az.hdi(local_series, 0.95)
                lower_hpd_log_95 = hpd_95[0]
                upper_hpd_log_95 = hpd_95[1]
                hpd_50 = az.hdi(local_series, 0.50)
                lower_hpd_log_50 = hpd_50[0]
                upper_hpd_log_50 = hpd_50[1]

                try:
                    local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, "median_percent":median_percent,
                                                       "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                       "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                    new_df = new_df.append(local_df)
                except:
                    pass
            count+=1

    return(new_df)

In [1411]:

log_file_path = "../simulations/beast_results/simulation_3/1_simulation_3_AllGenomes_SkygrowthPriorNoCases.log"

In [1412]:
#read in log file
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

#remove burn-in
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))

burnin_percent = 0.3
print(len(Ne_df))
rows_to_remove = int(len(Ne_df)* burnin_percent)
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()

ne_summary = generate_summary_df(Ne_df)

#format time intervals
test = ne_summary
test['days'] = (test.interval.astype(int))*7
test['date'] = dt.strptime("2023-01-14",  "%Y-%m-%d") - test.days.map(timedelta)

test["new_interval"] = (((test.interval.iloc[::-1]).astype(int))*7 )-7

#add smoothing 
columns = ["mean_Ne_log","mean_Ne_linear",  "median_Ne_linear","median_Ne_log", "upper_hpd_log_50","lower_hpd_log_50", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
for column in columns:
    # Create a new column for the moving average
    test[f'{column}_MA'] = test[column].rolling(5, min_periods =1).mean()


5001
5001
3501


In [1413]:
test

,interval,mean_Ne_log,mean_Ne_linear,median_Ne_log,median_Ne_linear,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,...,mean_Ne_log_MA,mean_Ne_linear_MA,median_Ne_linear_MA,median_Ne_log_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA,upper_hpd_linear_50_MA,lower_hpd_linear_50_MA,lower_hpd_linear_MA,upper_hpd_linear_MA
0,1,-1.131406,0.887760,-1.178720,0.307672,1.381923,-3.508764,-0.455971,-2.003911,3.982553,...,-1.131406,0.887760,0.307672,-1.178720,-0.455971,-2.003911,0.633832,0.134807,0.029934,3.982553
0,2,-1.054796,0.572553,-1.091342,0.335766,0.761844,-2.851836,-0.501293,-1.681841,2.142222,...,-1.093101,0.730156,0.321719,-1.135031,-0.478632,-1.842876,0.619790,0.160419,0.043836,3.062387
0,3,-0.971615,0.489766,-0.997963,0.368629,0.385510,-2.254991,-0.634393,-1.516377,1.470364,...,-1.052606,0.650026,0.337356,-1.089342,-0.530552,-1.734043,0.589946,0.180115,0.064182,2.531713
0,4,-0.857241,0.492347,-0.873826,0.417352,0.171477,-1.884198,-0.597808,-1.296939,1.187056,...,-1.003765,0.610607,0.357355,-1.035463,-0.547366,-1.624767,0.579963,0.203428,0.086124,2.195549
0,5,-0.706665,0.546189,-0.721922,0.485818,0.195945,-1.531285,-0.467567,-1.044116,1.216460,...,-0.944345,0.597723,0.383047,-0.972754,-0.531406,-1.508637,0.589275,0.233143,0.112151,1.999731
0,6,-0.551742,0.624649,-0.570875,0.565031,0.210472,-1.327681,-0.376249,-0.893146,1.234260,...,-0.828412,0.545101,0.434519,-0.851185,-0.515462,-1.286484,0.599795,0.288055,0.159182,1.450072
0,7,-0.436975,0.694066,-0.452030,0.636335,0.336423,-1.128237,-0.173823,-0.664044,1.399931,...,-0.704848,0.569403,0.494633,-0.723323,-0.449968,-1.082924,0.646735,0.353801,0.212356,1.301614
0,8,-0.360046,0.747532,-0.368456,0.691802,0.398907,-1.017685,-0.128607,-0.611156,1.490195,...,-0.582534,0.620957,0.559267,-0.597422,-0.348811,-0.901880,0.716548,0.418445,0.263667,1.305581
0,9,-0.309587,0.785845,-0.321229,0.725257,0.407187,-1.030348,-0.099884,-0.566049,1.502585,...,-0.473003,0.679656,0.620848,-0.486902,-0.249226,-0.755702,0.787533,0.477324,0.304653,1.368686
0,10,-0.270262,0.817079,-0.283135,0.753418,0.430315,-0.989870,-0.034678,-0.496174,1.537742,...,-0.385722,0.733834,0.674369,-0.399145,-0.162648,-0.646114,0.855411,0.528695,0.335727,1.432943


In [1414]:
#plot
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
log_line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.8, clip = True).encode(
    alt.X('new_interval', ),
    alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Effective Population Size", grid=False), scale = alt.Scale(domain=[-3,8])),
    alt.Y2('lower_hpd_log_50_MA' ),
    
).properties(
    width=850,
    height=300
)
log_band_ne = alt.Chart(test).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
    alt.X('new_interval', ),
    alt.Y('median_Ne_log_MA',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),

    
).properties(
    width=850,
    height=300
)

log_line_ne  + log_band_ne

alt.LayerChart(...)

In [1415]:
ne_diff_summary = generate_summary_diff_df(Ne_df)

uninfectious_rate = 365/4.5

#taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
incubation_period = 365/8

seir_transmission_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)
seir_transmission_rate.head()

,Ne.diff.1,Ne.diff.2,Ne.diff.3,Ne.diff.4,Ne.diff.5,Ne.diff.6,Ne.diff.7,Ne.diff.8,Ne.diff.9,Ne.diff.10,...,Ne.diff.26,Ne.diff.27,Ne.diff.28,Ne.diff.29,Ne.diff.30,Ne.diff.31,Ne.diff.32,Ne.diff.33,Ne.diff.34,Ne.diff.35
0,63.648481,63.797087,62.600756,63.594325,62.800270,64.008408,67.588224,81.608614,89.614341,97.997225,...,76.636642,81.311995,92.698619,108.069439,123.103396,129.530850,125.369737,118.392529,107.031835,27.341476
1,88.657509,94.691522,90.142679,76.152554,59.971079,60.039550,63.547386,70.898087,76.311428,92.399890,...,67.134368,70.555231,91.332595,95.487650,106.192434,115.322992,132.696202,138.681328,125.724256,58.416419
2,105.763800,93.863264,86.127103,80.952096,73.082481,69.563984,69.712848,73.665696,77.225330,84.570853,...,83.438538,85.265894,92.749125,105.164000,117.731786,130.479758,139.825746,133.285996,105.529197,14.770717
3,80.504454,75.736732,70.306377,62.912759,61.666731,59.168641,63.070503,65.524898,75.327358,84.003624,...,69.668076,80.815240,90.913347,102.073289,105.374180,116.894087,123.269249,131.773176,123.440957,85.550303
4,45.198522,47.622092,51.543965,54.379534,66.247707,71.645332,80.426019,80.330375,91.853771,101.130968,...,68.814380,82.380065,91.135167,106.804632,116.542962,121.362697,130.692533,118.563195,104.561969,49.015937


In [1416]:
simulation_df = pd.read_json("../simulations/simulation_results/eir_3.json", orient = "index")
simulation_df = simulation_df.T
simulation_df.I[simulation_df.I < 0.1] = 0.1
simulation_df.I = simulation_df.I.astype("float")
simulation_df["log_I"] = simulation_df.I.apply(np.log)
simulation_df["samp_diff"] = simulation_df.samp.diff()
simulation_df["samp_diff"][simulation_df["samp_diff"] < 0.1 ] = 0.1
simulation_df.loc[0,"samp_diff"] = 0.1
#simulation_df["log_samp_diff"] = simulation_df["samp_diff"].apply(np.log)
simulation_df.samp.value_counts()

463.0    43
466.0    35
52.0     35
94.0     32
232.0    29
         ..
358.0     1
359.0     1
361.0     1
42.0      1
260.0     1
Name: samp, Length: 529, dtype: int64

In [1417]:
simulation_df.loc[0,"samp_diff"]

0.1

In [1418]:
#add smoothing 
columns = ["samp_diff"]
for column in columns:
    # Create a new column for the moving average
    simulation_df[f'{column}_MA'] = simulation_df[column].rolling(21, min_periods =1).mean()

In [1419]:
simulation_df["new_time"] = simulation_df.t *366
line_simulation = alt.Chart(simulation_df).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "orange").encode(
    alt.X('new_time', axis=alt.Axis(title="")),
    alt.Y('samp_diff_MA',axis=alt.Axis(title="Simulated prevalence", grid=False)),
    
    
).properties(
    width=850,
    height=300)
line_simulation

alt.Chart(...)

In [1420]:
(line_simulation + log_line_ne + log_band_ne).resolve_scale(x = "shared", y = "shared")

alt.LayerChart(...)

In [1421]:
simulation_df_copy = simulation_df.copy()

simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)

daily_sim_cases = simulation_df_copy.groupby(["whole_time"]).agg({'log_I':'mean'}).reset_index()


#daily_sim_cases["diff"] = daily_sim_cases.log_I.diff() *52

#daily_sim_cases["diff"][daily_sim_cases["diff"] < 0 ] = 0

#define to week number
counter = 0
daily_sim_cases["week_time"] = np.nan
for index, row in daily_sim_cases.iterrows():
    if row.whole_time %7 == 0:
        daily_sim_cases.loc[index,"week_time"] = counter 
        counter +=1
        
#backfill
daily_sim_cases = daily_sim_cases.bfill()

# aggregate based on weeks
weekly_sim_cases = daily_sim_cases.groupby(["week_time"])["log_I"].mean().reset_index()

weekly_sim_cases["diff"] = weekly_sim_cases.log_I.diff() *7

weekly_sim_cases.head()

,week_time,log_I,diff
0,0.0,-2.302585,NaN
1,1.0,1.750675,28.372823
2,2.0,2.620298,6.087359
3,3.0,3.177530,3.900626
4,4.0,3.124837,-0.368853


In [1422]:
test = test.sort_values(by = ["new_interval"])
combinded_ne_and_sims = pd.concat([weekly_sim_cases, test.set_index(weekly_sim_cases.index)], axis=1)

In [1423]:
combinded_ne_and_sims

,week_time,log_I,diff,interval,mean_Ne_log,mean_Ne_linear,median_Ne_log,median_Ne_linear,upper_hpd_log_95,lower_hpd_log_95,...,mean_Ne_log_MA,mean_Ne_linear_MA,median_Ne_linear_MA,median_Ne_log_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA,upper_hpd_linear_50_MA,lower_hpd_linear_50_MA,lower_hpd_linear_MA,upper_hpd_linear_MA
0,0.0,-2.302585,NaN,38,-7.932820,1.364575,-7.980567,0.000342,-0.315408,-15.382042,...,-3.790721,0.335348,0.058380,-3.788980,-3.120535,-4.557108,0.077527,0.046492,0.027355,0.269572
1,1.0,1.750675,28.372823,37,-3.390471,0.039654,-3.358626,0.034783,-2.244107,-4.547029,...,-2.508713,0.107867,0.101874,-2.497690,-2.204142,-2.734352,0.130950,0.083462,0.052333,0.199760
2,2.0,2.620298,6.087359,36,-3.021601,0.054448,-3.007104,0.049435,-2.073011,-3.940835,...,-2.046207,0.171613,0.162147,-2.044007,-1.797750,-2.261107,0.199510,0.130248,0.086720,0.301643
3,3.0,3.177530,3.900626,35,-2.563512,0.082705,-2.556691,0.077561,-1.847449,-3.319263,...,-1.590045,0.262481,0.245682,-1.594824,-1.390977,-1.820343,0.294991,0.192311,0.130778,0.460366
4,4.0,3.124837,-0.368853,34,-2.045203,0.135357,-2.041911,0.129780,-1.474458,-2.651232,...,-1.185113,0.371578,0.343744,-1.196657,-1.004358,-1.431306,0.416919,0.267128,0.180473,0.666732
5,5.0,3.259275,0.941067,33,-1.522776,0.227172,-1.524118,0.217813,-0.966449,-2.080334,...,-0.870826,0.478922,0.439314,-0.887913,-0.689100,-1.137597,0.538291,0.338874,0.228509,0.893998
6,6.0,3.347525,0.617751,32,-1.077945,0.358380,-1.090212,0.336145,-0.485425,-1.700807,...,-0.668021,0.563216,0.513582,-0.688904,-0.488421,-0.963679,0.631535,0.390117,0.261523,1.071464
7,7.0,3.118725,-1.601603,31,-0.740790,0.508791,-0.761188,0.467111,-0.084011,-1.428287,...,-0.576646,0.607655,0.552539,-0.597487,-0.376343,-0.866973,0.690819,0.422221,0.274722,1.170783
8,8.0,2.965657,-1.071477,30,-0.538853,0.628189,-0.565859,0.567872,0.173507,-1.256465,...,-0.582683,0.605916,0.551510,-0.599701,-0.352350,-0.854659,0.704821,0.426651,0.271272,1.187529
9,9.0,3.067834,0.715241,29,-0.473766,0.672078,-0.498189,0.607630,0.311325,-1.168794,...,-0.664197,0.564020,0.515869,-0.675023,-0.426450,-0.932705,0.660684,0.398446,0.249292,1.107586


In [1424]:
chart = alt.Chart(combinded_ne_and_sims).mark_circle().encode(
        alt.X('log_I'),
        y='median_Ne_log'
).transform_filter(datum.log_I >2)

final = chart + chart.transform_regression('log_I','median_Ne_log').mark_line() + chart.transform_regression('log_I','median_Ne_log', groupby=["sampling"], params = True).mark_text().encode(
    x=alt.value(150),  # pixels from left
    y=alt.value(50),  # pixels from top
    text='params:N',
).transform_calculate(
    params='"r = " + round(sqrt(datum.rSquared) * 100)/100 ')

final
#chart +text+  chart.transform_regression('log_I','median_Ne_log_MA').mark_line()


alt.LayerChart(...)

In [1425]:
## calculate Rt

In [1426]:
rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
summary_rt_local_and_intro_df["new_interval"] = (((summary_rt_local_and_intro_df.interval.iloc[::-1]).astype(int))+.5)

In [1427]:
columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(4, min_periods =1, center = True).mean()

In [1428]:
summary_rt_local_and_intro_df

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,mean_percent_MA,median_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,1,0.908594,0.847528,1.907693,0.018781,1.081115,0.494469,35.5,0.881428,0.832408,1.758709,0.064855,1.049088,0.515550
0,2,0.854262,0.817288,1.609725,0.110928,1.017061,0.536630,34.5,0.856827,0.815361,1.636256,0.115060,1.013314,0.531374
0,3,0.807625,0.781268,1.391351,0.215472,0.941767,0.563022,33.5,0.842875,0.807593,1.553877,0.165053,0.990324,0.550846
0,4,0.801019,0.784288,1.306740,0.315033,0.921355,0.609264,32.5,0.824171,0.801145,1.396666,0.261304,0.959812,0.596098
0,5,0.833775,0.821738,1.278850,0.403785,0.959066,0.675475,31.5,0.831631,0.816229,1.323933,0.358518,0.955124,0.647961
0,6,0.884105,0.877621,1.318793,0.499781,0.998307,0.744082,30.5,0.860120,0.849742,1.308956,0.436808,0.975504,0.700813
0,7,0.921578,0.915322,1.331443,0.528634,1.023288,0.774432,29.5,0.897768,0.890291,1.322377,0.498708,1.013316,0.753236
0,8,0.951614,0.946483,1.360420,0.562632,1.072604,0.818954,28.5,0.936943,0.930905,1.354499,0.555779,1.048825,0.796248
0,9,0.990472,0.984194,1.407341,0.632070,1.101103,0.847524,27.5,0.980240,0.974049,1.389805,0.605583,1.089155,0.840083
0,10,1.057297,1.050196,1.460015,0.698994,1.159623,0.919423,26.5,1.032828,1.026311,1.436535,0.660866,1.141971,0.893799


In [1429]:
# simulation_df_copy = simulation_df.copy()
# simulation_df_copy["diff"] = simulation_df_copy.log_I.diff() *52
# simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)



In [1430]:
# simulation_df["whole_time"] = np.ceil(simulation_df.t *366)

# daily_sim_cases = simulation_df.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()


# daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
# daily_sim_cases["diff_I"] = daily_sim_cases.I.diff()


# daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
# daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

# counter = 0
# daily_sim_cases["week_time"] = np.nan
# for index, row in daily_sim_cases.iterrows():
#     if row.whole_time %7 == 0:
#         daily_sim_cases.loc[index,"week_time"] = counter 
#         counter +=1

# daily_sim_cases = daily_sim_cases.bfill()

# weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()

In [1431]:

simulation_df_copy = simulation_df.copy()

simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)

daily_sim_cases = simulation_df_copy.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()


daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
daily_sim_cases["diff_I"] = daily_sim_cases.I.diff() 

daily_sim_cases["log_I"] = daily_sim_cases.I.apply(np.log)
daily_sim_cases["diff"] = daily_sim_cases.I.diff()

#daily_sim_cases["log_diff"] = daily_sim_cases["diff"].apply(np.log) *365/7

#daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
#daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

#define to week number
counter = 0
daily_sim_cases["week_time"] = np.nan
for index, row in daily_sim_cases.iterrows():
    if row.whole_time %7 == 0:
        daily_sim_cases.loc[index,"week_time"] = counter 
        counter +=1
        
#backfill
daily_sim_cases = daily_sim_cases.bfill()

# aggregate based on weeks
weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()
#weekly_sim_cases["log_diff"] = weekly_sim_cases["log_diff"].div(7)

#weekly_sim_cases["I_no_intro"] = weekly_sim_cases.I - weekly_sim_cases.diff_intro
weekly_sim_cases["log_I"] = weekly_sim_cases.I.apply(np.log)
weekly_sim_cases["diff"] = weekly_sim_cases.log_I.diff() *(365/7)
#weekly_sim_cases["log_diff"] = weekly_sim_cases["diff"].apply(np.log) *36


weekly_sim_cases.head()

,week_time,diff_intro,I,log_I,diff
0,0.0,3.000000,0.100000,-2.302585,NaN
1,1.0,1.857143,6.408163,1.857573,216.922513
2,2.0,1.542857,14.260876,2.657520,41.711531
3,3.0,1.987755,24.365410,3.193165,27.930043
4,4.0,1.261054,23.046363,3.137508,-2.902092


In [1432]:
weekly_sim_cases

,week_time,diff_intro,I,log_I,diff
0,0.0,3.000000,0.100000,-2.302585,NaN
1,1.0,1.857143,6.408163,1.857573,216.922513
2,2.0,1.542857,14.260876,2.657520,41.711531
3,3.0,1.987755,24.365410,3.193165,27.930043
4,4.0,1.261054,23.046363,3.137508,-2.902092
5,5.0,2.401190,27.374550,3.309614,8.974088
6,6.0,1.714706,29.363576,3.379755,3.657364
7,7.0,1.928416,22.706887,3.122668,-13.405235
8,8.0,1.973545,19.552883,2.973123,-7.797732
9,9.0,3.190476,21.685242,3.076632,5.397266


In [1433]:
incubation_period = 365/8
uninfectious_rate = 365/4.5

weekly_sim_cases["Rt"] = (1+ (weekly_sim_cases["diff"] / uninfectious_rate)) * (1+ (weekly_sim_cases["diff"] / incubation_period))
weekly_sim_cases.week_time = (weekly_sim_cases.week_time -1.5) #*7
weekly_sim_cases = weekly_sim_cases.iloc[3:]

In [1434]:
columns = ["Rt"]
for column in columns:
    # Create a new column for the moving average
    weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(4, min_periods =1).mean()


In [1435]:
weekly_sim_cases

,week_time,diff_intro,I,log_I,diff,Rt,Rt_MA
3,1.5,1.987755,24.365410,3.193165,27.930043,2.167303,2.167303
4,2.5,1.261054,23.046363,3.137508,-2.902092,0.902889,1.535096
5,3.5,2.401190,27.374550,3.309614,8.974088,1.329094,1.466429
6,4.5,1.714706,29.363576,3.379755,3.657364,1.128867,1.382038
7,5.5,1.928416,22.706887,3.122668,-13.405235,0.589475,0.987581
8,6.5,1.973545,19.552883,2.973123,-7.797732,0.749385,0.949205
9,7.5,3.190476,21.685242,3.076632,5.397266,1.192709,0.915109
10,8.5,1.873016,12.648961,2.537575,-28.107966,0.250887,0.695614
11,9.5,1.448413,14.096782,2.645947,5.650798,1.202149,0.848783
12,10.5,2.307143,13.710111,2.618134,-1.450244,0.950902,0.899162


In [1436]:

line_rt_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "blue").encode(
    alt.X('week_time', axis=alt.Axis(title="")),
    alt.Y('Rt_MA',axis=alt.Axis(title="Rt", grid=False), scale = alt.Scale(domain = [0.5, 3])),
    
    
).properties(
    width=850,
    height=300)
line_rt_sim

alt.Chart(...)

In [1437]:
summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,mean_percent_MA,median_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,1,0.908594,0.847528,1.907693,0.018781,1.081115,0.494469,35.5,0.881428,0.832408,1.758709,0.064855,1.049088,0.515550
0,2,0.854262,0.817288,1.609725,0.110928,1.017061,0.536630,34.5,0.856827,0.815361,1.636256,0.115060,1.013314,0.531374
0,3,0.807625,0.781268,1.391351,0.215472,0.941767,0.563022,33.5,0.842875,0.807593,1.553877,0.165053,0.990324,0.550846
0,4,0.801019,0.784288,1.306740,0.315033,0.921355,0.609264,32.5,0.824171,0.801145,1.396666,0.261304,0.959812,0.596098
0,5,0.833775,0.821738,1.278850,0.403785,0.959066,0.675475,31.5,0.831631,0.816229,1.323933,0.358518,0.955124,0.647961


In [1438]:
band_rt = alt.Chart(summary_rt_local_and_intro_df).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "orange").encode(
    alt.X('new_interval', axis=alt.Axis(title="")),
    alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Rt", grid=False)),
    alt.Y2("lower_hpd_log_50_MA")
    
    
).properties(
    width=850,
    height=300)

line_rt = alt.Chart(summary_rt_local_and_intro_df).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
    alt.X('new_interval', axis=alt.Axis(title="")),
    alt.Y('median_percent_MA',axis=alt.Axis(title="Rt", grid=False)),

    
    
).properties(
    width=850,
    height=300)

(band_rt + line_rt+ line_rt_sim).resolve_scale(y ="shared")

alt.LayerChart(...)

In [1439]:
weekly_sim_cases

,week_time,diff_intro,I,log_I,diff,Rt,Rt_MA
3,1.5,1.987755,24.365410,3.193165,27.930043,2.167303,2.167303
4,2.5,1.261054,23.046363,3.137508,-2.902092,0.902889,1.535096
5,3.5,2.401190,27.374550,3.309614,8.974088,1.329094,1.466429
6,4.5,1.714706,29.363576,3.379755,3.657364,1.128867,1.382038
7,5.5,1.928416,22.706887,3.122668,-13.405235,0.589475,0.987581
8,6.5,1.973545,19.552883,2.973123,-7.797732,0.749385,0.949205
9,7.5,3.190476,21.685242,3.076632,5.397266,1.192709,0.915109
10,8.5,1.873016,12.648961,2.537575,-28.107966,0.250887,0.695614
11,9.5,1.448413,14.096782,2.645947,5.650798,1.202149,0.848783
12,10.5,2.307143,13.710111,2.618134,-1.450244,0.950902,0.899162


In [1440]:
summary_rt_local_and_intro_df = summary_rt_local_and_intro_df.sort_values(by = ["new_interval"])

In [1441]:
#weekly_sim_cases = weekly_sim_cases.iloc[:-1]
#weekly_sim_cases = weekly_sim_cases.iloc[:-2]
summary_rt_local_and_intro_df = summary_rt_local_and_intro_df.sort_values(by = ["new_interval"])
combinded_ne_and_sims = pd.concat([weekly_sim_cases, summary_rt_local_and_intro_df.set_index(weekly_sim_cases.index)], axis=1)

In [1442]:
combinded_ne_and_sims

,week_time,diff_intro,I,log_I,diff,Rt,Rt_MA,interval,mean_percent,median_percent,...,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,mean_percent_MA,median_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
3,1.5,1.987755,24.365410,3.193165,27.930043,2.167303,2.167303,35,6.049010,5.382001,...,-0.357813,6.666830,1.690760,1.5,3.261107,3.010459,6.337505,0.696172,3.536848,1.586969
4,2.5,1.261054,23.046363,3.137508,-2.902092,0.902889,1.535096,34,1.815875,1.778413,...,1.155641,1.930358,1.490464,2.5,2.922841,2.723341,5.400511,0.851830,3.153116,1.590506
5,3.5,2.401190,27.374550,3.309614,8.974088,1.329094,1.466429,33,1.918437,1.870962,...,1.290687,2.013356,1.579684,3.5,1.856643,1.816543,2.539659,1.264509,1.956251,1.547452
6,4.5,1.714706,29.363576,3.379755,3.657364,1.128867,1.382038,32,1.908042,1.861988,...,1.318806,2.001920,1.601115,4.5,1.796569,1.761412,2.432282,1.260120,1.891481,1.517069
7,5.5,1.928416,22.706887,3.122668,-13.405235,0.589475,0.987581,31,1.784220,1.754807,...,1.292900,1.879369,1.518544,5.5,1.652973,1.626639,2.221001,1.171287,1.750382,1.414675
8,6.5,1.973545,19.552883,2.973123,-7.797732,0.749385,0.949205,30,1.575578,1.557892,...,1.138085,1.671278,1.368934,6.5,1.459016,1.444059,1.953016,1.021419,1.565179,1.264798
9,7.5,3.190476,21.685242,3.076632,5.397266,1.192709,0.915109,29,1.344052,1.331869,...,0.935357,1.448960,1.170105,7.5,1.254127,1.246782,1.682895,0.834184,1.365838,1.091330
10,8.5,1.873016,12.648961,2.537575,-28.107966,0.250887,0.695614,28,1.132216,1.131665,...,0.719332,1.261110,1.001610,8.5,1.074159,1.072249,1.464894,0.665608,1.193325,0.931031
11,9.5,1.448413,14.096782,2.645947,5.650798,1.202149,0.848783,27,0.964662,0.965702,...,0.543961,1.082004,0.824670,9.5,0.934833,0.937249,1.297364,0.532294,1.063914,0.810266
12,10.5,2.307143,13.710111,2.618134,-1.450244,0.950902,0.899162,26,0.855706,0.859759,...,0.463780,0.981227,0.727737,10.5,0.839908,0.843198,1.194273,0.453633,0.965126,0.718698


In [1443]:

chart = alt.Chart(combinded_ne_and_sims).mark_circle().encode(
        alt.X('Rt_MA'),
        y='median_percent_MA'
)

final = chart + chart.transform_regression('Rt_MA','median_percent_MA').mark_line() + chart.transform_regression('Rt_MA','median_percent_MA', groupby=["sampling"], params = True).mark_text().encode(
    x=alt.value(150),  # pixels from left
    y=alt.value(50),  # pixels from top
    text='params:N',
).transform_calculate(
    params='"rSquared = " + round((datum.rSquared) * 100)/100 ')

final
#chart +text+  chart.transform_regression('log_I','median_Ne_log_MA').mark_line()


alt.LayerChart(...)

In [1444]:
## calculate percentage of cases due to introduction

In [1445]:
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [1446]:
def generate_percent_intro_df(input_df, seir_transmission_rate):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
        
            interval = i.split(".")[1]
            #deme = i.split(".")[2]

            try:
                temp_df["total."+ str(interval)] = seir_transmission_rate["Ne.diff." + str(interval)].astype("float") +  input_df[i].astype("float").apply(np.exp)
                new_df["intro.percent"+ "." + str(interval)] = input_df[i].astype("float").apply(np.exp).div(temp_df["total."+ str(interval)], axis = 0) 

            except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
                pass 
                  
    return(new_df)

In [1447]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df_mig(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            interval = i.split(".")[2]
            #interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_percent = np.median(local_series)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({ "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [1448]:
migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

#remove burnin
burnin_percent = 0.3
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
mig_df_f.head()

5001
3501


,index,sample,immigrationRate.1,immigrationRate.2,immigrationRate.3,immigrationRate.4,immigrationRate.5,immigrationRate.6,immigrationRate.7,immigrationRate.8,...,immigrationRate.28,immigrationRate.29,immigrationRate.30,immigrationRate.31,immigrationRate.32,immigrationRate.33,immigrationRate.34,immigrationRate.35,immigrationRate.36,immigrationRate.37
0,1500,75000000,3.113340471012345,3.378249174127799,3.25271777292982,2.7442168277006918,2.429867045928879,2.6829913108851735,2.787212449227552,2.6960835158536445,...,3.1050353562605975,3.0387369338924954,2.49954210078923,2.497329020912626,2.9021317999785357,3.106980563651298,3.0736710273480417,3.1667605508712504,3.397225729264085,3.4440478708150315
1,1501,75050000,2.937903294417808,2.548190587822086,2.3390603379455186,2.767956105324777,3.2689684142291933,3.184378924355249,3.019966554563034,2.508915878684802,...,2.766470736641271,2.8971100060189054,2.3942899507495627,2.9213009522213857,2.967945224242808,2.8130007618241275,2.963088431636831,3.122153592350304,3.347317666380638,3.5158214384652258
2,1502,75100000,3.02173440677537,3.2475477815742093,3.4919242406618403,3.408669468800222,3.019161604018973,2.4379182676538513,2.313444005339927,2.929179163495228,...,3.0147772421471477,2.051268565975917,2.90976681968356,2.9812969597282026,2.867002322653348,2.526181491755575,2.5439001800905117,3.9121199394260247,3.619237356846726,3.3971928761907546
3,1503,75150000,3.264888764340174,3.258380198404606,2.5068798322055006,2.6852431594574617,2.34728294446879,2.3794908240783252,2.605522422256759,2.2693358345640036,...,2.6268722674859135,2.776065879831494,2.6864422506076897,3.1360259213038675,2.9912951183192336,3.0345662565875133,3.0186196534566467,3.392853509522099,3.480417380670669,3.1921928012355743
4,1504,75200000,3.1479008469762677,2.9816366267177457,3.049926378955493,2.870756404113138,2.3029990808356637,2.2559150158827994,2.639133751278414,2.3623207478676527,...,2.935246967150561,2.981566616081532,2.5802228975758545,3.1334177202887865,3.4354731523090627,2.5178250377641405,3.1138776305200677,3.5368905918918454,2.6240776021825214,3.008558088873656


In [1449]:
seir_transmission_rate.head()

,Ne.diff.1,Ne.diff.2,Ne.diff.3,Ne.diff.4,Ne.diff.5,Ne.diff.6,Ne.diff.7,Ne.diff.8,Ne.diff.9,Ne.diff.10,...,Ne.diff.26,Ne.diff.27,Ne.diff.28,Ne.diff.29,Ne.diff.30,Ne.diff.31,Ne.diff.32,Ne.diff.33,Ne.diff.34,Ne.diff.35
0,63.648481,63.797087,62.600756,63.594325,62.800270,64.008408,67.588224,81.608614,89.614341,97.997225,...,76.636642,81.311995,92.698619,108.069439,123.103396,129.530850,125.369737,118.392529,107.031835,27.341476
1,88.657509,94.691522,90.142679,76.152554,59.971079,60.039550,63.547386,70.898087,76.311428,92.399890,...,67.134368,70.555231,91.332595,95.487650,106.192434,115.322992,132.696202,138.681328,125.724256,58.416419
2,105.763800,93.863264,86.127103,80.952096,73.082481,69.563984,69.712848,73.665696,77.225330,84.570853,...,83.438538,85.265894,92.749125,105.164000,117.731786,130.479758,139.825746,133.285996,105.529197,14.770717
3,80.504454,75.736732,70.306377,62.912759,61.666731,59.168641,63.070503,65.524898,75.327358,84.003624,...,69.668076,80.815240,90.913347,102.073289,105.374180,116.894087,123.269249,131.773176,123.440957,85.550303
4,45.198522,47.622092,51.543965,54.379534,66.247707,71.645332,80.426019,80.330375,91.853771,101.130968,...,68.814380,82.380065,91.135167,106.804632,116.542962,121.362697,130.692533,118.563195,104.561969,49.015937


In [1450]:
percent_df = generate_percent_intro_df(mig_df_f, seir_transmission_rate)
percent_df.head()



,intro.percent.1,intro.percent.2,intro.percent.3,intro.percent.4,intro.percent.5,intro.percent.6,intro.percent.7,intro.percent.8,intro.percent.9,intro.percent.10,...,intro.percent.26,intro.percent.27,intro.percent.28,intro.percent.29,intro.percent.30,intro.percent.31,intro.percent.32,intro.percent.33,intro.percent.34,intro.percent.35
0,0.261143,0.314868,0.292337,0.196501,0.153152,0.186029,0.193688,0.153703,0.136290,0.143496,...,0.229629,0.264735,0.193986,0.161916,0.090012,0.085756,0.126846,0.158821,0.168058,0.464648
1,0.175538,0.118948,0.103184,0.172961,0.304726,0.286872,0.243826,0.147754,0.113327,0.108204,...,0.187655,0.212078,0.148295,0.159509,0.093557,0.138663,0.127849,0.107247,0.133426,0.279802
2,0.162537,0.215127,0.276098,0.271864,0.218842,0.141325,0.126647,0.202562,0.141613,0.147873,...,0.199656,0.149686,0.180181,0.068865,0.134861,0.131253,0.111710,0.085778,0.107639,0.771971
3,0.245377,0.255616,0.148555,0.189002,0.144988,0.154348,0.176720,0.128634,0.118860,0.127704,...,0.137495,0.137828,0.132041,0.135917,0.122273,0.164483,0.139065,0.136282,0.142199,0.258027
4,0.340029,0.292834,0.290592,0.245042,0.131199,0.117553,0.148274,0.116724,0.126310,0.123567,...,0.173507,0.214871,0.171207,0.155850,0.101740,0.159043,0.191952,0.094694,0.177132,0.412109


In [1451]:
summary_percent_df = generate_summary_df_mig(percent_df)
summary_percent_df = summary_percent_df.reset_index()



In [1452]:
summary_percent_df

,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50
0,0,1,0.269177,0.606214,0.049690,0.332115,0.138078
1,0,2,0.247017,0.492630,0.061095,0.298244,0.147788
2,0,3,0.208786,0.393619,0.067040,0.245806,0.136545
3,0,4,0.191341,0.354215,0.075938,0.223260,0.130107
4,0,5,0.169210,0.296102,0.069538,0.193723,0.117408
5,0,6,0.152197,0.258670,0.068685,0.178320,0.112260
6,0,7,0.146924,0.251278,0.062448,0.170777,0.106879
7,0,8,0.148133,0.254617,0.068393,0.180482,0.118006
8,0,9,0.144977,0.242808,0.060402,0.165124,0.105448
9,0,10,0.144999,0.244761,0.067065,0.168173,0.110768


In [1453]:
#making sure that any numbers >1 are excluded as outliers from stochastic noise
percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

#format dates
summary_percent_df["new_interval"] = (((summary_percent_df.interval.iloc[::-1]).reset_index(drop=True).astype(int))+.5)   #smoothing
summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
summary_percent_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)

summary_percent_df.head()





,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,days,date
0,0,1,0.269177,0.606214,0.049690,0.332115,0.138078,35.5,-14,2024-09-26
1,0,2,0.247017,0.492630,0.061095,0.298244,0.147788,34.5,-7,2024-09-19
2,0,3,0.208786,0.393619,0.067040,0.245806,0.136545,33.5,0,2024-09-12
3,0,4,0.191341,0.354215,0.075938,0.223260,0.130107,32.5,7,2024-09-05
4,0,5,0.169210,0.296102,0.069538,0.193723,0.117408,31.5,14,2024-08-29


In [1454]:
summary_percent_df.interval

0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
20    21
21    22
22    23
23    24
24    25
25    26
26    27
27    28
28    29
29    30
30    31
31    32
32    33
33    34
34    35
Name: interval, dtype: object

In [1455]:
columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(4, min_periods =1).mean()


In [1456]:
summary_percent_df.head()

,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,days,date,mean_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,0,1,0.269177,0.606214,0.049690,0.332115,0.138078,35.5,-14,2024-09-26,0.269177,0.606214,0.049690,0.332115,0.138078
1,0,2,0.247017,0.492630,0.061095,0.298244,0.147788,34.5,-7,2024-09-19,0.258097,0.549422,0.055393,0.315180,0.142933
2,0,3,0.208786,0.393619,0.067040,0.245806,0.136545,33.5,0,2024-09-12,0.241660,0.497488,0.059275,0.292055,0.140804
3,0,4,0.191341,0.354215,0.075938,0.223260,0.130107,32.5,7,2024-09-05,0.229080,0.461669,0.063441,0.274856,0.138129
4,0,5,0.169210,0.296102,0.069538,0.193723,0.117408,31.5,14,2024-08-29,0.204088,0.384141,0.068403,0.240258,0.132962


In [1457]:
line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('new_interval'),
    alt.Y('lower_hpd_log_50_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_50_MA' )
).properties(
    width=850,
    height=300
)

band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('new_interval'),
    alt.Y('lower_hpd_log_95_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_95_MA' )
).properties(
    width=80,
    height=300
)

#line1
intro_plot = (line1 + band).resolve_scale(y= "shared") ; intro_plot

alt.LayerChart(...)

In [1458]:
## now calculate the true percentage of cases due to intros from simulations

In [1459]:
simulation_df["whole_time"] = np.ceil(simulation_df.t *366)

In [1460]:
daily_sim_cases = simulation_df.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()


In [1461]:
daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
daily_sim_cases["diff_I"] = daily_sim_cases.I.diff()


daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

In [1462]:
counter = 0
daily_sim_cases["week_time"] = np.nan
for index, row in daily_sim_cases.iterrows():
    if row.whole_time %7 == 0:
        daily_sim_cases.loc[index,"week_time"] = counter 
        counter +=1

In [1463]:
daily_sim_cases = daily_sim_cases.bfill()

In [1464]:
weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()

In [1465]:
weekly_sim_cases["percent_intro"] = weekly_sim_cases.diff_intro/(weekly_sim_cases.diff_intro+ weekly_sim_cases.I)

In [1466]:
weekly_sim_cases.week_time = (weekly_sim_cases.week_time -1.5) #*7
weekly_sim_cases = weekly_sim_cases.iloc[3:]

In [1467]:
weekly_sim_cases.head()

,week_time,diff_intro,I,percent_intro
3,1.5,1.987755,24.365410,0.075428
4,2.5,1.261054,23.046363,0.051879
5,3.5,2.401190,27.374550,0.080643
6,4.5,1.714706,29.363576,0.055174
7,5.5,1.928416,22.706887,0.078279


In [1468]:
columns = ["percent_intro"]
for column in columns:
    # Create a new column for the moving average
    weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(3, min_periods =1, center = True).mean()

In [1469]:
weekly_sim_cases.head()

,week_time,diff_intro,I,percent_intro,percent_intro_MA
3,1.5,1.987755,24.365410,0.075428,0.063653
4,2.5,1.261054,23.046363,0.051879,0.069316
5,3.5,2.401190,27.374550,0.080643,0.062565
6,4.5,1.714706,29.363576,0.055174,0.071365
7,5.5,1.928416,22.706887,0.078279,0.075044


In [1470]:
percent_intro_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "black").encode(
    alt.X('week_time', axis=alt.Axis(title="")),
    alt.Y('percent_intro_MA',axis=alt.Axis(title="percent of cases due to introductions", grid=False)),
    
    
).properties(
    width=850,
    height=300)

percent_intro_sim


alt.Chart(...)

In [1471]:
intro_plot + percent_intro_sim

alt.LayerChart(...)

In [1472]:
summary_percent_df = summary_percent_df.sort_values(by = ["new_interval"])

#weekly_sim_cases = weekly_sim_cases.iloc[:-1]
#weekly_sim_cases = weekly_sim_cases.iloc[:-2]

combinded_ne_and_sims = pd.concat([weekly_sim_cases, summary_percent_df.set_index(weekly_sim_cases.index)], axis=1)

combinded_ne_and_sims




,week_time,diff_intro,I,percent_intro,percent_intro_MA,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,days,date,mean_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
3,1.5,1.987755,24.365410,0.075428,0.063653,0,35,0.081106,0.351190,0.013098,0.087720,0.030415,1.5,224,2024-02-01,0.111960,0.235670,0.047322,0.128060,0.079404
4,2.5,1.261054,23.046363,0.051879,0.069316,0,34,0.142446,0.226707,0.069688,0.165609,0.114597,2.5,217,2024-02-08,0.120843,0.194485,0.057511,0.139155,0.093863
5,3.5,2.401190,27.374550,0.080643,0.062565,0,33,0.112266,0.180635,0.050493,0.128895,0.084967,3.5,210,2024-02-15,0.116265,0.187969,0.055038,0.133534,0.089044
6,4.5,1.714706,29.363576,0.055174,0.071365,0,32,0.112024,0.184148,0.056008,0.130017,0.087637,4.5,203,2024-02-22,0.124847,0.202942,0.059802,0.144131,0.096413
7,5.5,1.928416,22.706887,0.078279,0.075044,0,31,0.116638,0.186448,0.053856,0.132100,0.088253,5.5,196,2024-02-29,0.137396,0.220953,0.064055,0.158398,0.105968
8,6.5,1.973545,19.552883,0.091680,0.099405,0,30,0.124131,0.200643,0.059794,0.143125,0.095320,6.5,189,2024-03-07,0.156784,0.250604,0.074749,0.180879,0.121251
9,7.5,3.190476,21.685242,0.128257,0.116305,0,29,0.146596,0.240528,0.069550,0.171282,0.114443,7.5,182,2024-03-14,0.186053,0.293756,0.089269,0.214788,0.146137
10,8.5,1.873016,12.648961,0.128978,0.116803,0,28,0.162220,0.256195,0.073020,0.187085,0.125858,8.5,175,2024-03-21,0.225891,0.352281,0.111241,0.260763,0.179979
11,9.5,1.448413,14.096782,0.093174,0.122064,0,27,0.194188,0.305050,0.096632,0.222024,0.149381,9.5,168,2024-03-28,0.263663,0.414240,0.137372,0.301205,0.208233
12,10.5,2.307143,13.710111,0.144041,0.130062,0,26,0.241208,0.373253,0.117873,0.278761,0.194867,10.5,161,2024-04-04,0.281881,0.444456,0.147985,0.321759,0.221680


In [1473]:
chart = alt.Chart(combinded_ne_and_sims).mark_circle().encode(
        alt.X('percent_intro_MA'),
        y='mean_percent_MA'
)

final = chart + chart.transform_regression('percent_intro_MA','mean_percent_MA').mark_line() + chart.transform_regression('percent_intro_MA','mean_percent_MA', groupby=["sampling"], params = True).mark_text().encode(
    x=alt.value(150),  # pixels from left
    y=alt.value(50),  # pixels from top
    text='params:N',
).transform_calculate(
    params='"rSquared = " + round((datum.rSquared) * 100)/100 ')

final
#chart +text+  chart.transform_regression('log_I','median_Ne_log_MA').mark_line()


alt.LayerChart(...)

In [1381]:
def plot_ne_and_rt_comparison(log_file_path, simulation_path):
    # read in log file for changes in Ne
    Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

    # remove burn-in
    Ne_df = pd.DataFrame.from_dict(Ne_skyline)

    burnin_percent = 0.2
    print(len(Ne_df))
    rows_to_remove = int(len(Ne_df)* burnin_percent)
    Ne_df = Ne_df.iloc[rows_to_remove:]

    print(len(Ne_df))
    Ne_df = Ne_df.reset_index()
    
#     ne_summary = generate_summary_df(Ne_df)

#     # format time intervals
#     test = ne_summary
#     test['days'] = (test.interval.astype(int))*7
#     test['date'] = dt.strptime("2023-01-14",  "%Y-%m-%d") - test.days.map(timedelta)
#     test["new_interval"] = (((test.interval.iloc[::-1]).astype(int))*7 )-7

#     # add smoothing 
#     columns = ["mean_Ne_log","mean_Ne_linear",  "median_Ne_linear","median_Ne_log", "upper_hpd_log_50","lower_hpd_log_50", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
#     for column in columns:
#         # Create a new column for the moving average
#         test[f'{column}_MA'] = test[column].rolling(4, min_periods =1).mean()


#     # plot the Ne over time for sims analyzed via multi-tree coal
#     range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]

#     log_line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.8, clip = True).encode(
#         alt.X('new_interval', ),
#         alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="", grid=False), scale = alt.Scale(domain=[-3,8])),
#         alt.Y2('lower_hpd_log_50_MA' ),    
#     ).properties(
#         width=850,
#         height=300
#     )

#     log_band_ne = alt.Chart(test).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
#         alt.X('new_interval', ),
#         alt.Y('median_Ne_log_MA',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),   
#     ).properties(
#         width=850,
#         height=300
#     )



    ## calculate SEIR transmission rate using the uninfectious period and the incubation period. 
    uninfectious_rate = 365/4.5 #taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
    incubation_period = 365/8

    ne_diff_summary = generate_summary_diff_df(Ne_df) #summarize data in easier datatable format
    seir_transmission_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)


    ## read in simulation data directly from MASTER json. 
    simulation_df = pd.read_json(simulation_path, orient = "index")
    simulation_df = simulation_df.T

    ## log transform for easier comparison with BEAST output
    simulation_df.I[simulation_df.I < 0.1] = 0.1
    simulation_df.I = simulation_df.I.astype("float")
    simulation_df["log_I"] = simulation_df.I.apply(np.log)
    simulation_df["new_time"] = simulation_df.t *366


    #plot the log cases from simulations
    line_simulation = alt.Chart(simulation_df).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "orange").encode(
        alt.X('new_time', axis=alt.Axis(title="time interval")),
        alt.Y('log_I',axis=alt.Axis(title="Simulated prevalence", grid=False)),    
    ).properties(
        width=850,
        height=300)


    final_ne_vs_sim_plot = (line_simulation + log_line_ne + log_band_ne).resolve_scale(x = "shared", y = "shared")

    ####### calculate Rt from BEAST 

    rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
    summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
    summary_rt_local_and_intro_df["new_interval"] = (((summary_rt_local_and_intro_df.interval.iloc[::-1]).astype(int))+.5)    
    columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
    for column in columns:
        # Create a new column for the moving average
        summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(4, min_periods =1, center = True).mean()


    ## plot BEAST Rt
    band_rt = alt.Chart(summary_rt_local_and_intro_df).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "orange").encode(
        alt.X('new_interval', axis=alt.Axis(title="time interval")),
        alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Rt", grid=False)),
        alt.Y2("lower_hpd_log_50_MA")  
    ).properties(
        width=550,
        height=200)

    line_rt = alt.Chart(summary_rt_local_and_intro_df).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
        alt.X('new_interval', axis=alt.Axis(title="time interval")),
        alt.Y('median_percent_MA',axis=alt.Axis(title="Rt", grid=False)),
    ).properties(
        width=550,
        height=200)

    ###### calculate Rt from simulations
    simulation_df_copy = simulation_df.copy()

    simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)

    daily_sim_cases = simulation_df_copy.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()


    daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
    daily_sim_cases["diff_I"] = daily_sim_cases.I.diff() 

    daily_sim_cases["log_I"] = daily_sim_cases.I.apply(np.log)
    daily_sim_cases["diff"] = daily_sim_cases.I.diff()

    #daily_sim_cases["log_diff"] = daily_sim_cases["diff"].apply(np.log) *365/7

    #daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
    #daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

    #define to week number
    counter = 0
    daily_sim_cases["week_time"] = np.nan
    for index, row in daily_sim_cases.iterrows():
        if row.whole_time %7 == 0:
            daily_sim_cases.loc[index,"week_time"] = counter 
            counter +=1

    #backfill
    daily_sim_cases = daily_sim_cases.bfill()

    # aggregate based on weeks
    weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()

    weekly_sim_cases["log_I"] = weekly_sim_cases.I.apply(np.log)
    weekly_sim_cases["diff"] = weekly_sim_cases.log_I.diff() *(365/7)


    incubation_period = 365/8
    uninfectious_rate = 365/4.5

    weekly_sim_cases["Rt"] = (1+ (weekly_sim_cases["diff"] / uninfectious_rate)) * (1+ (weekly_sim_cases["diff"] / incubation_period))
    weekly_sim_cases.week_time = (weekly_sim_cases.week_time -1.5) #*7
    weekly_sim_cases = weekly_sim_cases.iloc[3:]
    

    columns = ["Rt"]
    for column in columns:
        # Create a new column for the moving average
        weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(4, min_periods =1).mean()

    summary_rt_local_and_intro_df = summary_rt_local_and_intro_df.sort_values(by = ["new_interval"])
    combinded_ne_and_sims = pd.concat([weekly_sim_cases, summary_rt_local_and_intro_df.set_index(weekly_sim_cases.index)], axis=1)


    #plot Rt from simulations
    line_rt_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "blue").encode(
        alt.X('week_time', axis=alt.Axis(title="")),
        alt.Y('Rt_MA',axis=alt.Axis(title="Rt", grid=False), scale = alt.Scale(domain = [0.3, 3])),   
    ).properties(
        width=550,
        height=200)

    final_rt_vs_sim_plot = (band_rt + line_rt+ line_rt_sim).resolve_scale(y ="shared")
    
    
    chart = alt.Chart(combinded_ne_and_sims).mark_circle().encode(
            x = alt.X('Rt_MA', axis=alt.Axis(title="Simulation Rt")),
            y = alt.Y('median_percent', axis=alt.Axis(title="Estimated Rt"))
    )

    final_rt_corr = chart + chart.transform_regression('Rt_MA','median_percent').mark_line() + chart.transform_regression('Rt_MA','median_percent', groupby=["sampling"], params = True).mark_text().encode(
        x=alt.value(150),  # pixels from left
        y=alt.value(50),  # pixels from top
        text='params:N',
    ).transform_calculate(
        params='"rSquared = " + round((datum.rSquared) * 100)/100 '
    ).properties(
        width=550,
        height=200)


    return final_rt_vs_sim_plot, final_rt_corr

In [1479]:
simulation_path = "../simulations/simulation_results/eir_3.json"
#log_file_path = "../simulations/beast_results/simmulticoal_4_all.log"

In [1500]:
import os

folder_path = '../simulations/beast_results/simulation_3/'

# List all files in sims folder
log_files = [(folder_path + file) for file in os.listdir(folder_path) if file.endswith('.log')]

#put everything in order
log_files.sort()

In [1501]:
chart_list_ne = []
chart_list_rt = []

for file in log_files:
    base_filename = file.split("/")[3].split(".")[0]
    
    plot1, plot2 = plot_ne_and_rt_comparison(file, simulation_path)
    plot1 = plot1.properties(title="Rt values - estimated vs simulation:" + file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])
    plot2 = plot2.properties(title="Rt correlation- estimated vs simulation:" + file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])

    
    chart_list_ne.append(plot1)
    chart_list_rt.append(plot2)

5001
4001
5001
4001
28184
22548
48667
38934
18196
14557
31787
25430


In [1502]:
plots_1 = alt.vconcat(*chart_list_ne)

In [1503]:
plots_2 = alt.vconcat(*chart_list_rt)

In [1504]:
plots_1 | plots_2

alt.HConcatChart(...)

In [1530]:
def plot_percent_intro_comparison(log_file_path, simulation_path):

    ### now working on percentage of cases due to introductions
    Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

    # remove burn-in
    Ne_df = pd.DataFrame.from_dict(Ne_skyline)

    burnin_percent = 0.3
    print(len(Ne_df))
    rows_to_remove = int(len(Ne_df)* burnin_percent)
    Ne_df = Ne_df.iloc[rows_to_remove:]

    print(len(Ne_df))
    Ne_df = Ne_df.reset_index()
    

    ## calculate SEIR transmission rate using the uninfectious period and the incubation period. 
    uninfectious_rate = 365/4.5 #taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
    incubation_period = 365/8

    ne_diff_summary = generate_summary_diff_df(Ne_df) #summarize data in easier datatable format
    seir_transmission_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)

    # read in migration rate estimates from bEAST
    migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
    mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

    # remove burnin
    burnin_percent = 0.3
    #print(len(mig_df_f))
    rows_to_remove = int(len(mig_df_f)* burnin_percent)
    mig_df_f = mig_df_f.iloc[rows_to_remove:]

    #print(len(mig_df_f))
    mig_df_f = mig_df_f.reset_index()

    # calculate % of cases due to introd
    percent_df = generate_percent_intro_df(mig_df_f, seir_transmission_rate)

    # summarize data in easier to use format
    summary_percent_df = generate_summary_df_mig(percent_df)
    summary_percent_df = summary_percent_df.reset_index()

    # making sure that any numbers >1 are excluded as outliers from stochastic noise
    percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
    percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

    # format dates
    summary_percent_df["new_interval"] = (((summary_percent_df.interval.iloc[::-1]).reset_index(drop=True).astype(int))+.5)   #smoothing
    summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
    summary_percent_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)
    #print(summary_percent_df)
    # smoothing due to stochasitc nosie
    columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
    for column in columns:
        # Create a new column for the moving average
        summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(3, min_periods =1).mean()

    #print(summary_percent_df)
    # plot % of cases due to intros from BEAST
    line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
        alt.X('new_interval', title = "Time Interval"),
        alt.Y('lower_hpd_log_50_MA',title = "Percent of cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
        alt.Y2('upper_hpd_log_50_MA' )
    ).properties(
        width=550,
        height=200)

    band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
        alt.X('new_interval', title = "Time Interval"),
        alt.Y('lower_hpd_log_95_MA',title = "Percent of cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
        alt.Y2('upper_hpd_log_95_MA' )
    ).properties(
        width=550,
        height=200)

    #line1
    intro_plot = (line1 + band).resolve_scale(y= "shared") 
    
    ## read in simulation dynamics
    simulation_df = pd.read_json(simulation_path, orient = "index")
    simulation_df = simulation_df.T

    ## log transform for easier comparison with BEAST output
    simulation_df.I[simulation_df.I < 0.1] = 0.1
    simulation_df.I = simulation_df.I.astype("float")
    simulation_df["log_I"] = simulation_df.I.apply(np.log)
    simulation_df["new_time"] = simulation_df.t *366

    ## now calculate the true percentage of cases due to intros from simulations
    simulation_df["whole_time"] = np.ceil(simulation_df.t *366)

    # aggregate to whole time due to the simulations having multiple events in the same day
    daily_sim_cases = simulation_df.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()

    # take the difference to calcualte the incidence and new intros since I refers to the prevalence and not new cases
    daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
    daily_sim_cases["diff_I"] = daily_sim_cases.I.diff()


    daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
    daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

    # changing daily to week
    counter = 0
    daily_sim_cases["week_time"] = np.nan
    for index, row in daily_sim_cases.iterrows():
        if row.whole_time %7 == 0:
            daily_sim_cases.loc[index,"week_time"] = counter 
            counter +=1
    daily_sim_cases = daily_sim_cases.bfill()
    weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()

    # calculate % of cases due to intros from sims
    weekly_sim_cases["percent_intro"] = weekly_sim_cases.diff_intro/(weekly_sim_cases.diff_intro+ weekly_sim_cases.I)

    weekly_sim_cases.week_time = (weekly_sim_cases.week_time -1.5) #*7
    weekly_sim_cases = weekly_sim_cases.iloc[3:]
    
    #smoothing - rolling mean 
    columns = ["percent_intro"]
    for column in columns:
        # Create a new column for the moving average
        weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(3, min_periods =1, center = True).mean()

    
    #plot from sims
    percent_intro_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "black").encode(
        alt.X('week_time', axis=alt.Axis(title="Time Interval")),
        alt.Y('percent_intro_MA',axis=alt.Axis(title="Percent of cases due to introductions", grid=False)),
    ).properties(
        width=550,
        height=200)

    final_plot = line1 +percent_intro_sim
    
        
    summary_percent_df = summary_percent_df.sort_values(by = ["new_interval"])

    combinded_percent_and_sims = pd.concat([weekly_sim_cases, summary_percent_df.set_index(weekly_sim_cases.index)], axis=1)

    
    chart = alt.Chart(combinded_percent_and_sims).mark_circle().encode(
        alt.X('percent_intro_MA', title = "Truth via Simulations"),
        alt.Y('mean_percent_MA', title = "Estimated via BEAST "
    ))

    final = chart + chart.transform_regression('percent_intro_MA','mean_percent_MA').mark_line() + chart.transform_regression('percent_intro_MA','mean_percent_MA', groupby=["sampling"], params = True).mark_text().encode(
        x=alt.value(150),  # pixels from left
        y=alt.value(50),  # pixels from top
        text='params:N',
    ).transform_calculate(
        params='"rSquared = " + round((datum.rSquared) * 100)/100 ').properties(
        width=550,
        height=200)

    
    return final_plot, final

In [1535]:
chart_list_percent_intro = []
corr_list_plot = []

for file in log_files:
    # Get the base filename without the extension
    base_filename = file.split("/")[3].split(".")[0]
    
    # Assign the output of plot_ne_and_rt_comparison to variables
    plot1, plot2 = plot_percent_intro_comparison(file,simulation_path )
    
    plot1 = plot1.properties(title="% of cases due to imports - estimated vs simulation:" +file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])
    plot2 = plot2.properties(title="% of cases due to imports correlation - estimated vs simulation:" +file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])


  
    chart_list_percent_intro.append(plot1)
    corr_list_plot.append(plot2)


5001
3501
5001
3501
28184
19729
48667
34067
18196
12738
31787
22251


In [1536]:
percent_sim_plots = alt.vconcat(*chart_list_percent_intro)

In [1537]:
percent_sims_corr = alt.vconcat(*corr_list_plot)

In [1538]:
percent_sim_plots | percent_sims_corr

alt.HConcatChart(...)